Alice and Bob have an undirected graph of `n` nodes and three types of edges:
- Type 1: Can be traversed by Alice only.
- Type 2: Can be traversed by Bob only.
- Type 3: Can be traversed by both Alice and Bob.

Given an array `edges` where **edges[i] = [type<sub>i</sub>, u<sub>i</sub>, v<sub>i</sub>]** represents a bidirectional edge of type **type<sub>i</sub>** between nodes **u<sub>i</sub>** and **v<sub>i</sub>**, find the maximum number of edges you can remove so that after removing the edges, the graph can still be fully traversed by both Alice and Bob. The graph is fully traversed by Alice and Bob if starting from any node, they can reach all other nodes.

Return *the maximum number of edges you can remove, or return* `-1` *if Alice and Bob cannot fully traverse the graph*.

<br>

**Example 1:**

![ex5](../../images/ex5.png)

>**Input:** n = 4, edges = [[3,1,2],[3,2,3],[1,1,3],[1,2,4],[1,1,2],[2,3,4]]<br>
>**Output:** 2<br>
>**Explanation:** If we remove the 2 edges [1,1,2] and [1,1,3]. The graph will still be fully traversable by Alice and Bob. Removing any additional edge will not make it so. So the maximum number of edges we can remove is 2.

**Example 2:**

![ex6](../../images/ex6.png)

>**Input:** n = 4, edges = [[3,1,2],[3,2,3],[1,1,4],[2,1,4]]<br>
>**Output:** 0<br>
>**Explanation:** Notice that removing any edge will not make the graph fully traversable by Alice and Bob.

**Example 3:**

![ex7](../../images/ex7.png)

>**Input:** n = 4, edges = [[3,2,3],[1,1,2],[2,3,4]]<br>
>**Output:** -1<br>
>**Explanation:** In the current graph, Alice cannot reach node 4 from the other nodes. Likewise, Bob cannot reach 1. Therefore it's impossible to make the graph fully traversable.

<br>

**Constraints:**
- >1 <= n <= 10<sup>5</sup>
- >1 <= edges.length <= min(10<sup>5</sup>, 3 * n * (n - 1) / 2)
- >edges[i].length == 3
- >1 <= type<sub>i</sub> <= 3
- >1 <= u<sub>i</sub> < v<sub>i</sub> <= n
- >All tuples (type<sub>i</sub>, u<sub>i</sub>, v<sub>i</sub>) are distinct.

In [1]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [1] * n

    def find(self, u):
        if self.parent[u] != u:
            self.parent[u] = self.find(self.parent[u])
        return self.parent[u]

    def union(self, u, v):
        root_u = self.find(u)
        root_v = self.find(v)
        if root_u == root_v:
            return False
        if self.rank[root_u] > self.rank[root_v]:
            self.parent[root_v] = root_u
        elif self.rank[root_u] < self.rank[root_v]:
            self.parent[root_u] = root_v
        else:
            self.parent[root_v] = root_u
            self.rank[root_u] += 1
        return True


class Solution:
    def maxNumEdgesToRemove(self, n: int, edges: list[list[int]]) -> int:
        edges.sort(reverse=True, key=lambda x: x[0])
    
        uf_common = UnionFind(n)
        uf_alice = UnionFind(n)
        uf_bob = UnionFind(n)
        
        num_edges_used = 0
        
        for edge in edges:
            t, u, v = edge
            u -= 1
            v -= 1
            if t == 3:
                if uf_common.union(u, v):
                    uf_alice.union(u, v)
                    uf_bob.union(u, v)
                    num_edges_used += 1
        
        for edge in edges:
            t, u, v = edge
            u -= 1
            v -= 1
            if t == 1:
                if uf_alice.union(u, v):
                    num_edges_used += 1
            elif t == 2:
                if uf_bob.union(u, v):
                    num_edges_used += 1
        
        root_alice = uf_alice.find(0)
        root_bob = uf_bob.find(0)
        if all(uf_alice.find(i) == root_alice for i in range(n)) and all(uf_bob.find(i) == root_bob for i in range(n)):
            return len(edges) - num_edges_used
        else:
            return -1